In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver import ActionChains
import pandas as pd
import numpy as np
import time

In [2]:
def get_driver():
    options = webdriver.FirefoxOptions()
    # options.headless = True
    driver = webdriver.Firefox(options = options, service=Service(GeckoDriverManager().install()))
    return driver

In [156]:
def get_data():
    driver = get_driver()
    url = "https://finder.startupnationcentral.org/startups/search"
    driver.get(url)

    # driver.execute_script("window.scrollTo(0, Y)") 

    # time.sleep(3)
    # loadmore = driver.find_element(By.XPATH , "/html/body/div[25]/div[2]/div[2]/div/div[4]")
    # # ActionChains(driver).move_to_element(loadmore).perform()
    # loadmore.click()


    body = driver.find_element(By.CLASS_NAME,"company-cards-list")
    companies = body.find_elements(By.CLASS_NAME,"box-view-item")
    links = []
    for company in companies:
        links.append(company.find_element(By.TAG_NAME,"a").get_attribute("href"))
        
    driver.quit()
    return links




In [13]:
def load_more(driver):
    SCROLL_PAUSE_TIME = 0.5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [26]:
def load_all_page():
    driver = get_driver()
    url = "https://finder.startupnationcentral.org/startups/search"
    driver.get(url)
    
    # load_more(driver)

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    loadmore = driver.find_element(By.CLASS_NAME , "js-search-load-more-button")
    ActionChains(driver).move_to_element(loadmore).perform()
    loadmore.click()

In [28]:
load_all_page()



====== WebDriver manager ======
Current firefox version is 99.0
Get LATEST geckodriver version for 99.0 firefox
Driver [C:\Users\matan\.wdm\drivers\geckodriver\win64\v0.31.0\geckodriver.exe] found in cache


In [157]:
links = get_data()



====== WebDriver manager ======
Current firefox version is 99.0
Get LATEST geckodriver version for 99.0 firefox
Driver [C:\Users\matan\.wdm\drivers\geckodriver\win64\v0.31.0\geckodriver.exe] found in cache


In [158]:
len(links)

30

In [159]:
links

['https://finder.startupnationcentral.org/company_page/hypervision',
 'https://finder.startupnationcentral.org/company_page/golan-plastic-products',
 'https://finder.startupnationcentral.org/company_page/chakratec',
 'https://finder.startupnationcentral.org/company_page/zohar-cleantech',
 'https://finder.startupnationcentral.org/company_page/annabella',
 'https://finder.startupnationcentral.org/company_page/algoretail',
 'https://finder.startupnationcentral.org/company_page/sunbit',
 'https://finder.startupnationcentral.org/company_page/rewalk-robotics',
 'https://finder.startupnationcentral.org/company_page/laminate-medical',
 'https://finder.startupnationcentral.org/company_page/cham-food',
 'https://finder.startupnationcentral.org/company_page/persimio',
 'https://finder.startupnationcentral.org/company_page/weeedrive',
 'https://finder.startupnationcentral.org/company_page/appsvillage',
 'https://finder.startupnationcentral.org/company_page/core-platform',
 'https://finder.startupn

In [146]:
def get_clasiffications(page):
    """Get the classification list for a given page\n
    page is a WebDriver object.\n
    Return one line df"""

    name = page.find_element(By.CLASS_NAME,"top-profile-section").find_element(By.CLASS_NAME,"title").text
    about = page.find_element(By.CLASS_NAME,"about").text

    classifications = page.find_element(By.CLASS_NAME, "js-startup-classification-section").find_elements(By.CLASS_NAME,"classification-item")
    l = [np.nan,np.nan,np.nan]
    for idx, cls in enumerate(classifications):

        elements = cls.find_elements(By.CLASS_NAME,"js-lead-item")
        element_list = []
        for elm in elements:
            elm_title = elm.find_element(By.CLASS_NAME,"row-container").text
            element_list.append(elm_title)

        l[idx]= ", ".join(element_list)
    
    fund_data = page.find_element(By.CLASS_NAME, "funding-metadata").find_elements(By.CLASS_NAME,"title")
    fund_stage = fund_data[0].text
    total_raised = fund_data[1].text
    total_rounds = fund_data[2].text
    investors = fund_data[3].text




    d = {'company_name' : name, 'company_about': about, 'sector': l[0], 'target_ind':l[1], 'technology':l[2],
        'fund_stage':fund_stage,'total_raised':total_raised,'total_rounds':total_rounds,'investors': investors}

    return pd.DataFrame([d])
        



In [106]:
driver = get_driver()
driver.get(links[0])
# df = pd.DataFrame()
# df = get_clasiffications(driver, df)
# driver.quit()



====== WebDriver manager ======
Current firefox version is 99.0
Get LATEST geckodriver version for 99.0 firefox
Driver [C:\Users\matan\.wdm\drivers\geckodriver\win64\v0.31.0\geckodriver.exe] found in cache


In [147]:
df = pd.DataFrame(columns=['company_name','company_about','sector','target_ind','technology','fund_stage','total_raised','total_rounds','investors'])
driver = get_driver()
for link in links[:3]:
    driver.get(link)
    df = pd.concat([df,get_clasiffications(driver)], ignore_index=True)
df



====== WebDriver manager ======
Current firefox version is 99.0
Get LATEST geckodriver version for 99.0 firefox
Driver [C:\Users\matan\.wdm\drivers\geckodriver\win64\v0.31.0\geckodriver.exe] found in cache


,company_name,company_about,sector,target_ind,technology,fund_stage,total_raised,total_rounds,investors
0,Zooz Power,Zooz Power provides flywheel energy storage te...,"Energy-tech, Smart Mobility","Energy, Utilities & Waste Management, Governme...",NaN,Public,48.5M,5,5
1,ReWalk Robotics,ReWalk Robotics is a medical device company th...,Life Sciences & HealthTech,Healthcare & Life Sciences,"Communications, Machinery & Robotics, Semicond...",Public,112.8M,11,7
2,AppsVillage,AppsVillage has developed a platform that enab...,Retail & Marketing,"Commerce & Retail, Enterprise & Professional S...",Platforms & Interfaces,Public,1.3M,2,1


In [154]:
np.isnan(df.iloc[0,4])

True